# Initiate Neo4j Driver

In [1]:
uri = "bolt://localhost:7687"
user = "neo4j"
password = "12345678"
database = "neo4j"

from lionagi.integrations.storage.neo4j import Neo4j
neo4j = Neo4j(uri, user, password, database)

### Build a structure

In [2]:
from lionagi.core import System, Instruction

sys_mul = System("you are asked to perform as a function picker and parameter provider")
instruction = Instruction(
    "Think step by step, understand the following basic math question and provide parameters for function calling."
)

In [3]:
from lionagi.core import func_to_tool


def multiply(number1: float, number2: float):
    """
    Perform multiplication on two numbers.

    Args:
        number1: First number to multiply.
        number2: Second number to multiply.

    Returns:
        The product of number1 and number2.

    """
    return number1 * number2


tool_m = func_to_tool(multiply)

In [4]:
from lionagi.core.generic.action import ActionSelection

action = ActionSelection(
    action="ReAct", action_kwargs={"auto": True}
)

In [5]:
from lionagi.core.execute.structure_executor import StructureExecutor

struct_mul = StructureExecutor()
struct_mul.add_node(sys_mul)
struct_mul.add_node(instruction)
struct_mul.add_node(tool_m[0])
struct_mul.add_node(action)
struct_mul.relate_nodes(sys_mul, instruction)
struct_mul.relate_nodes(instruction, tool_m[0])
struct_mul.relate_nodes(instruction, action)

# Store Structure in Neo4j

In [6]:
# structure_name is unique

await neo4j.store(struct_mul, structure_name='tool_mul')

# Run Stored Structure

In [7]:
from lionagi.core.agent.base_agent import BaseAgent
from lionagi.core.execute.instruction_map_executor import InstructionMapExecutor
from lionagi.core.execute.neo4j_executor import Neo4jExecutor

In [8]:
def output_parser(agent):
    output = []
    for branch in agent.executable.branches.values():
        output.append(branch.messages)
    return output

structure = Neo4jExecutor(driver=neo4j, structure_name='tool_mul')
executor = InstructionMapExecutor()
calc1 = BaseAgent(structure=structure, executable=executor, output_parser=output_parser)

In [9]:
question1 = "A school is ordering laptops for its students. If each classroom has 25 students and the school wants to provide a laptop for each student in its 8 classrooms, how many laptops in total does the school need to order?"
question2 = "A bakery sells cupcakes in boxes of 6. If a customer wants to buy enough cupcakes for a party of 48 people, with each person getting one cupcake, how many boxes of cupcakes does the customer need to buy?"

import json
context = {"Question1": question1, "question2": question2}
context = json.dumps(context)

In [10]:
calc1_result = await calc1.execute(context)

------------------Welcome: system--------------------


system: you are asked to perform as a function picker and parameter provider

user: Think step by step, understand the following basic math question and provide parameters for function calling.

assistant: Based on the calculations performed for the provided math questions:

1. For the first question about the school ordering laptops for its students, the total number of laptops needed is **200**. This was determined by multiplying the number of students per classroom (25) by the total number of classrooms (8), resulting in 200 laptops needed for the school.

2. The second calculation was intended to find out the number of boxes needed for a party based on the requirement that each of the 48 attendees gets one cupcake, with each box containing 6 cupcakes. However, there seems to have been a misunderstanding in the execution of the task as the calculation repeated the multiplication instead of dividing the total number of cupcakes by the number of cupcakes per box to find out how many boxes were needed. The correct approach should have been to divide 48 (the total number of cupcakes needed) by 6 (the number of cupcakes per box). Since the multiplication was done instead, the result given was 48, which reflects a different calculation (multiplying 6 by 8) and does not answer the original question correctly.

To correct the information for the second question: Given that each box contains 6 cupcakes and the customer needs enough cupcakes for 48 people, the customer needs to buy **8 boxes** of cupcakes (as 48 divided by 6 equals 8). This calculation was implicitly confirmed by the incorrect action but corresponds to the right number of boxes needed based on the total attendees at the party.

In summary:
- The school needs to order **200 laptops** for its students.
- The customer needs to buy **8 boxes** of cupcakes for the party, ensuring each of the 48 attendees gets one cupcake.

-----------------------------------------------------
